In [1]:
!pip install transformers shap torch -q



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import shap
import numpy as np
import json

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("rk68/distilbert-q-classifier-3")
model = AutoModelForSequenceClassification.from_pretrained("rk68/distilbert-q-classifier-3")
model.eval()  # Set the model to evaluation mode

# Load the explanation dataset (JSON file)
file_path = "../giskard_test_sets/LL144_275_New.jsonl"
explanation_questions = []
with open(file_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        explanation_questions.append(data['question'])

# Tokenize the explanation questions
explanation_tokenized = tokenizer(explanation_questions, padding=True, truncation=True, return_tensors="pt")

# Select a representative subset of the explanation questions for SHAP analysis
subset_indices = np.random.choice(len(explanation_questions), size=50, replace=False)  # For example, 50 samples
subset_tokenized = {key: value[subset_indices] for key, value in explanation_tokenized.items()}

# Define the prediction function for SHAP
def predict_proba(inputs):
    # Move inputs to the appropriate device
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    
    # Get logits from the model
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Convert logits to probabilities using softmax
    probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()
    
    return probabilities

# Use the selected subset for SHAP's KernelExplainer
explainer = shap.KernelExplainer(predict_proba, subset_tokenized)

# Generate SHAP values for the subset of the explanation dataset
shap_values = explainer.shap_values(subset_tokenized)

# Visualize SHAP values for the first few samples
shap.summary_plot(shap_values, feature_names=tokenizer.convert_ids_to_tokens(subset_tokenized['input_ids'][0].tolist()))


TypeError: Unknown type passed as data object: <class 'dict'>

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import shap
import numpy as np
import json

# Ensure that `mps` is available
device = torch.device("mps") if torch.has_mps else torch.device("cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("rk68/distilbert-q-classifier-3")
model = AutoModelForSequenceClassification.from_pretrained("rk68/distilbert-q-classifier-3")
model.to(device)  # Move the model to the MPS device
model.eval()  # Set the model to evaluation mode

# Load only 10 samples from the explanation dataset (JSON file)
file_path = "../giskard_test_sets/LL144_275_New.jsonl"
explanation_questions = []
with open(file_path, 'r') as f:
    for i, line in enumerate(f):
        if i >= 10:  # Stop after 10 samples
            break
        data = json.loads(line)
        explanation_questions.append(data['question'])

# Tokenize the explanation questions
explanation_tokenized = tokenizer(explanation_questions, padding=True, truncation=True, return_tensors="pt")

# Define the prediction function for SHAP
def predict_proba(combined_inputs):
    # Determine the length of input_ids and attention_mask from the original data
    half = combined_inputs.shape[1] // 2
    input_ids = combined_inputs[:, :half]
    attention_mask = combined_inputs[:, half:]

    # Convert back to tensors and move to the `mps` device
    input_ids = torch.tensor(input_ids).to(device)
    attention_mask = torch.tensor(attention_mask).to(device)
    
    # Get logits from the model
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
    
    # Convert logits to probabilities using softmax and move back to CPU for SHAP processing
    probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()
    
    return probabilities

# Use the 10 samples from your combined inputs for SHAP's KernelExplainer
input_ids = explanation_tokenized['input_ids'].numpy()
attention_mask = explanation_tokenized['attention_mask'].numpy()

# Combine into a single array
combined_inputs = np.concatenate([input_ids, attention_mask], axis=1)

# Initialize KernelExplainer
explainer = shap.KernelExplainer(predict_proba, combined_inputs[:10])  # Use only 10 samples

# Generate SHAP values for the explanation dataset
shap_values = explainer.shap_values(combined_inputs[:10])

# Visualize SHAP values for the first few samples
shap.summary_plot(shap_values, feature_names=tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))


/var/folders/vh/rp6l70s568gc8_g2c6pn2lnc0000gn/T/ipykernel_1601/2239563485.py:8: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps") if torch.has_mps else torch.device("cpu")
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 16.17 GB, other allocations: 98.67 MB, max allowed: 18.13 GB). Tried to allocate 3.02 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).